### Covid-19 Data extraction using John Hopkins [repo](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/README.md)<br>
lat long of [countries](https://developers.google.com/public-data/docs/canonical/countries_csv)

In [1]:
import pandas as pd
import wget

save data to local from github using ```wget.download```

In [2]:
urls = ["https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv",
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv",
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv",
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
       ]

In [40]:
for url in urls:
    filename = wget.download(url)

100% [............................................................................] 842772 / 842772

In [3]:
global_confirmed    = pd.read_csv("time_series_covid19_confirmed_global.csv")
global_deaths       = pd.read_csv("time_series_covid19_deaths_global.csv")
global_recovered    = pd.read_csv("time_series_covid19_recovered_global.csv")
global_us_deaths    = pd.read_csv("time_series_covid19_deaths_US.csv")
global_us_confirmed = pd.read_csv("time_series_covid19_confirmed_US.csv")

In [4]:
lat_long_all = pd.read_csv("lat_long_countires__global_20200409.csv", encoding = "ISO-8859-1")
lat_long_all["name"] = lat_long_all["name"].str.lower()
lat_long_all.head()

,country,Lat,Long,name
0,AD,42.55,1.60,andorra
1,AE,23.42,53.85,united arab emirates
2,AF,33.94,67.71,afghanistan
3,AG,17.06,-61.80,antigua and barbuda
4,AI,18.22,-63.07,anguilla


In [5]:
display(global_us_deaths.head())
display(global_confirmed.head())

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,1,2,3,3,4,4,4,4,4,4
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,0,0,1,1,1,1,1,1,1,1
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,6,8,11,12,15,18,20,21,23,24
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,0,0,0,0,0,0,1,1,1,1


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,170,174,237,273,281,299,349,367,423,444
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,223,243,259,277,304,333,361,377,383,400
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,584,716,847,986,1171,1251,1320,1423,1468,1572
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,370,376,390,428,439,466,501,525,545,564
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,7,7,8,8,8,10,14,16,17,19


Data Cleaning

__Global__

In [6]:
## ship names 
province = ["Diamond Princess" "MS Zaandam"]

In [7]:
def wide_to_long(pd_df, value_name_):
    '''
    input - pandas df wide
    melt, the df and format datetime
    output - wide df 
    '''
    
    dates_columns = pd_df.columns[4:]
    
    pd_df.rename(columns = {"Province/State":"Province_State","Country/Region" :"Country_Region" }, inplace = True)
    
    melted_df = pd_df.melt(id_vars   = ["Province_State","Country_Region","Lat","Long"], 
                       value_vars = dates_columns,
                       var_name   = "date_new",
                       value_name = value_name_)
    
    melted_df["date_new"] = pd.to_datetime(melted_df.date_new, format = "%m/%d/%y")
    return(melted_df)

In [16]:
cleaned_global_confirmed     = wide_to_long(global_confirmed   , "confirmed")  
cleaned_global_deaths        = wide_to_long(global_deaths      , "deaths") 
cleaned_global_recovered     = wide_to_long(global_recovered   , "recovered")    

display(len(cleaned_global_confirmed))
display(len(cleaned_global_deaths))   
display(len(cleaned_global_recovered))

20514

20514

19422

In [18]:
## merge all global
## the merge is counter intuitive 
## the left is on the right actually 
global_all_1 = cleaned_global_deaths.merge( cleaned_global_confirmed  ,  on = ["Province_State","Country_Region","date_new"], 
                                              how = "left"
                                             )
global_all   =  global_all_1.merge( cleaned_global_recovered,  on = ["Province_State","Country_Region","date_new"], 
                                  how = "left")

display(len(global_all_1))
display(len(global_all))

20514

20514

In [19]:
sequence_cols = ["Province_State","Country_Region", "Lat", "Long", "date_new", "confirmed","deaths","recovered"]
global_all    = global_all[sequence_cols]

display(global_all.head())
display(len(global_all))

,Province_State,Country_Region,Lat,Long,date_new,confirmed,deaths,recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0.0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0.0


20514

In [20]:
## country province date level data 
global_all.to_csv("global_all_except_us.csv", index= False)

Generating data at country date level 

In [21]:
gloabl_groupby = global_all.groupby(["Country_Region", "date_new"]).agg({"confirmed":"sum", 
                                                        "deaths"   : "sum",
                                                        "recovered": "sum"}).reset_index()      

gloabl_groupby["Country_Region"] = gloabl_groupby["Country_Region"].str.lower()
gloabl_groupby.head()

,Country_Region,date_new,confirmed,deaths,recovered
0,afghanistan,2020-01-22,0,0,0.0
1,afghanistan,2020-01-23,0,0,0.0
2,afghanistan,2020-01-24,0,0,0.0
3,afghanistan,2020-01-25,0,0,0.0
4,afghanistan,2020-01-26,0,0,0.0


all NULL provinces have _LAT_ _LONG_

In [23]:
## where province is np.nan and lat is np.nan
global_all[(pd.isna(global_all["Province_State"]) & (pd.isna(global_all["Lat"])))]

,Province_State,Country_Region,Lat,Long,date_new,confirmed,deaths,recovered


In [25]:
## where province is not np.nan but lat is np.nan
global_all[( (pd.isna(global_all["Lat"])))][["Country_Region"]].drop_duplicates()

,Country_Region
35,Canada


Getting lat long 

In [26]:
print("Total countires " ,len(global_all.Country_Region.drop_duplicates()))

Total countires  184


For those countires having provinces or multiple lat long we, extract their value from lag long file.

In [28]:
global_all["Country_Region"] = global_all["Country_Region"].str.lower()

gloabl_all_lat_long_duplicates = global_all[(pd.notna(global_all["Province_State"]))][["Country_Region", "Lat"]].drop_duplicates().reset_index()
print_1 = gloabl_all_lat_long_duplicates.groupby(["Country_Region"]).agg({"Lat":pd.Series.nunique}).reset_index().sort_values(by ="Lat", ascending = False )
print(print_1)


gloabl_all_lat_long_duplicates = global_all[(pd.notna(global_all["Province_State"]))][["Country_Region"]].drop_duplicates().reset_index()
new_lat_long_for_duplicates = lat_long_all.merge( gloabl_all_lat_long_duplicates, right_on = ["Country_Region"], left_on = ["name"], how = "right").drop(["country", "index", "name"], axis = 1)
display(new_lat_long_for_duplicates)
display(len(new_lat_long_for_duplicates))

   Country_Region   Lat
2           china  33.0
4          france  10.0
6  united kingdom  10.0
0       australia   8.0
5     netherlands   4.0
3         denmark   2.0
1          canada   0.0


,Lat,Long,Country_Region
0,-25.27,133.78,australia
1,56.13,-106.35,canada
2,35.86,104.20,china
3,56.26,9.50,denmark
4,46.23,2.21,france
5,55.38,-3.44,united kingdom
6,52.13,5.29,netherlands


7

In [29]:
gloabl_all_lat_long = global_all[(pd.isna(global_all["Province_State"]))][["Country_Region", "Lat", "Long"]].drop_duplicates().reset_index()
gloabl_all_lat_long.drop(["index"], axis =1 , inplace = True)
gloabl_all_lat_long["Country_Region"] = gloabl_all_lat_long["Country_Region"].str.lower()
display(gloabl_all_lat_long.head())
display(len(gloabl_all_lat_long))

,Country_Region,Lat,Long
0,afghanistan,33.0000,65.0000
1,albania,41.1533,20.1683
2,algeria,28.0339,1.6596
3,andorra,42.5063,1.5218
4,angola,-11.2027,17.8739


181

In [30]:
remove_dumplicates = gloabl_all_lat_long.merge(new_lat_long_for_duplicates, on = ["Country_Region"], how = "inner")["Country_Region"]

new_lat_long_for_duplicates_final = new_lat_long_for_duplicates[new_lat_long_for_duplicates["Country_Region"].isin(remove_dumplicates) == False]

new_lat_long_for_duplicates_final = new_lat_long_for_duplicates_final[["Country_Region", "Lat", "Long"]]

In [31]:
final_lat_long = pd.concat([new_lat_long_for_duplicates_final, gloabl_all_lat_long])

if len(global_all.Country_Region.drop_duplicates()) == len(final_lat_long.Country_Region):
    print("proceed")

proceed


In [32]:
## ship names given 0,0 lat,long
final_lat_long[final_lat_long["Lat"] == 0]

,Country_Region,Lat,Long
36,diamond princess,0.0,0.0
173,ms zaandam,0.0,0.0


In [33]:
print("lenght of global merged = ", len(gloabl_groupby))
final_covid_tableau_df = final_lat_long.merge(gloabl_groupby, on = ["Country_Region"], how = "inner")
sequence_cols = ["Country_Region", "Lat", "Long", "date_new", "confirmed","deaths","recovered"]
final_covid_tableau_df = final_covid_tableau_df[sequence_cols]
display(final_covid_tableau_df.head())
print("Merged DF length")
display(len(final_covid_tableau_df))
final_covid_tableau_df.to_csv("final_covid_tableau_df.csv", index = False)

lenght of global merged =  14352


,Country_Region,Lat,Long,date_new,confirmed,deaths,recovered
0,australia,-25.27,133.78,2020-01-22,0,0,0.0
1,australia,-25.27,133.78,2020-01-23,0,0,0.0
2,australia,-25.27,133.78,2020-01-24,0,0,0.0
3,australia,-25.27,133.78,2020-01-25,0,0,0.0
4,australia,-25.27,133.78,2020-01-26,4,0,0.0


Merged DF length


14352

__US__

In [34]:
## US has extra columns to be removed
columns_to_drop_us = ["iso2","iso3", "code3", "FIPS","Admin2", "Combined_Key"]

## axis =1 for drop() remove columns
global_us_deaths.drop   (columns_to_drop_us, axis = 1,inplace = True)
global_us_confirmed.drop(columns_to_drop_us, axis = 1,inplace = True)

global_us_deaths.drop   ("Population", axis = 1,inplace = True)

display(global_us_deaths.head())

,UID,Province_State,Country_Region,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,16,American Samoa,US,-14.2710,-170.1320,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,316,Guam,US,13.4443,144.7937,0,0,0,0,0,...,1,2,3,3,4,4,4,4,4,4
2,580,Northern Mariana Islands,US,15.0979,145.6739,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,1
3,630,Puerto Rico,US,18.2208,-66.5901,0,0,0,0,0,...,6,8,11,12,15,18,20,21,23,24
4,850,Virgin Islands,US,18.3358,-64.8963,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1


In [35]:
def wide_to_long_us(pd_df, value_name_):
    '''
    input - pandas df wide
    melt, the df and format datetime
    output - wide df 
    '''
    
    dates_columns = pd_df.columns[5:]
    
    pd_df.rename(columns = {"Long_" :"Long" }, inplace = True)
    
    melted_df = pd_df.melt(id_vars   = ["UID","Province_State","Country_Region","Lat","Long"], 
                       value_vars = dates_columns,
                       var_name   = "date_new",
                       value_name = value_name_)
    
    melted_df["date_new"] = pd.to_datetime(melted_df.date_new, format = "%m/%d/%y")
    
    return(melted_df)

In [36]:
cleaned_global_us_confirmed  = wide_to_long_us(global_us_confirmed, "confirmed") 
cleaned_global_us_deaths     = wide_to_long_us(global_us_deaths   , "deaths")    
display(cleaned_global_us_deaths.head())
display(len(cleaned_global_us_confirmed))

,UID,Province_State,Country_Region,Lat,Long,date_new,deaths
0,16,American Samoa,US,-14.2710,-170.1320,2020-01-22,0
1,316,Guam,US,13.4443,144.7937,2020-01-22,0
2,580,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0
3,630,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0
4,850,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0


253734

In [37]:
us_merged = cleaned_global_us_deaths.merge( cleaned_global_us_confirmed, on = ["UID", "Province_State", "Country_Region", "date_new", "Lat", "Long"], how = "left")     

In [38]:
display(us_merged.head())
display(len(us_merged))

,UID,Province_State,Country_Region,Lat,Long,date_new,deaths,confirmed
0,16,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,NaN
1,316,Guam,US,13.4443,144.7937,2020-01-22,0,0.0
2,580,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,0.0
3,630,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,0.0
4,850,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,0.0


253734

In [39]:
display(us_merged.isnull().sum())
us_merged["confirmed"] = us_merged.confirmed.fillna(0)

UID                   0
Province_State        0
Country_Region        0
Lat                   0
Long                  0
date_new              0
deaths                0
confirmed         23166
dtype: int64

In [40]:
us_merged.to_csv("us_covid.csv", index = False)

To tableau data, Day level

In [41]:
us_merged_tableau  =us_merged.groupby(["Country_Region", "date_new"]).agg({"deaths" : "sum",
                                                                           "confirmed" : "sum"}).reset_index()


us_merged_tableau["recovered"] = None

display(us_merged_tableau.head())
display(len(us_merged_tableau))

,Country_Region,date_new,deaths,confirmed,recovered
0,US,2020-01-22,0,1.0,None
1,US,2020-01-23,0,1.0,None
2,US,2020-01-24,0,2.0,None
3,US,2020-01-25,0,2.0,None
4,US,2020-01-26,0,3.0,None


78

In [42]:
display(lat_long_all[lat_long_all["name"] == "united states"])

us_merged_tableau_lat_long =  us_merged_tableau.merge( lat_long_all, how = "left", left_on = "Country_Region", right_on = "country")

sequence_cols = ["Country_Region", "Lat", "Long", "date_new", "confirmed","deaths","recovered"]

us_day_level = us_merged_tableau_lat_long[sequence_cols]
display(us_day_level.head())
display(len(us_day_level))

,country,Lat,Long,name
227,US,37.09,-95.71,united states


,Country_Region,Lat,Long,date_new,confirmed,deaths,recovered
0,US,37.09,-95.71,2020-01-22,1.0,0,None
1,US,37.09,-95.71,2020-01-23,1.0,0,None
2,US,37.09,-95.71,2020-01-24,2.0,0,None
3,US,37.09,-95.71,2020-01-25,2.0,0,None
4,US,37.09,-95.71,2020-01-26,3.0,0,None


78

In [43]:
covid_19_df = pd.concat([us_day_level, final_covid_tableau_df], ignore_index = True)
display(covid_19_df.head())
covid_19_df.to_csv("tableau_covid_data.csv", index = False)

,Country_Region,Lat,Long,date_new,confirmed,deaths,recovered
0,US,37.09,-95.71,2020-01-22,1.0,0,None
1,US,37.09,-95.71,2020-01-23,1.0,0,None
2,US,37.09,-95.71,2020-01-24,2.0,0,None
3,US,37.09,-95.71,2020-01-25,2.0,0,None
4,US,37.09,-95.71,2020-01-26,3.0,0,None


Basic EDA on the Data

In [44]:
## shape of data
covid_19_df.shape

(14430, 7)

In [45]:
len(covid_19_df.groupby(["Country_Region", "date_new"]).agg({"confirmed":pd.Series.nunique}))

14430

Data Problems : - <br>
1. Lat long for all data not available
2. LOD -  province - country - date level
3. NULL is recovered.
4. Province data available but lat long not available
5. 0 - latitude longitude
6. US data has no recovery numbers 
7. merging provinces of global countries into one

__Visualization on Tableau__ check out the next lockdown-upload